In [1]:
# Loading packages

from datetime import datetime
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
# define fixed variables 

batch_size = 256
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [10]:
# Loading image arrays
with np.load("notmnist_small.npz") as f:
    data = f['data']
    labels = f['labels']

x_train, x_test, y_train, y_test = train_test_split(data, labels)

In [11]:
# Convert from 0-255 to 0-1
x_train /= 255
x_test /= 255

In [12]:
# Resize from (m, 28, 28) to (m, 28, 28, 1), since Keras always assumes the 4th
# dimension in case of RGB images, even if we're just doing B/W images.
x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [13]:
print("Found %s records. Splitting into %s training and %s test records." % (
    x_train.shape[0] + x_test.shape[0],
    x_train.shape[0],
    x_test.shape[0],
))


Found 18726 records. Splitting into 14044 training and 4682 test records.


In [14]:
# y_train is a 1-D array of labels 0-9
y_train

array([ 4.,  4.,  3., ...,  9.,  9.,  9.])

In [15]:
# Convert from numeric labels to one-hot encoding, to make it
# easier to work with a neural network
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [29]:
# This is a simpler model that can easily be trained on a laptop
model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

In [30]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [31]:
print("Model Summary:")
print(model.summary())

Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                173120    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650   

In [33]:
# Declare tensorboard callback
tensorboard = TensorBoard(log_dir='./logs/%s' % datetime.now().strftime('%Y-%m-%d_%H:%M'),
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)


In [34]:
# Train model on training set
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard])


Train on 14044 samples, validate on 4682 samples
Epoch 1/10
14044/14044 [==============================] - 5s - loss: 6.8888 - acc: 0.4727 - val_loss: 2.7539 - val_acc: 0.7439
Epoch 2/10
14044/14044 [==============================] - 4s - loss: 1.4722 - acc: 0.7740 - val_loss: 0.6404 - val_acc: 0.8449
Epoch 3/10
14044/14044 [==============================] - 5s - loss: 0.7474 - acc: 0.8190 - val_loss: 0.5223 - val_acc: 0.8563
Epoch 4/10
14044/14044 [==============================] - 5s - loss: 0.5603 - acc: 0.8480 - val_loss: 0.4540 - val_acc: 0.8678
Epoch 5/10
14044/14044 [==============================] - 5s - loss: 0.4739 - acc: 0.8645 - val_loss: 0.4210 - val_acc: 0.8776
Epoch 6/10
14044/14044 [==============================] - 5s - loss: 0.4153 - acc: 0.8815 - val_loss: 0.3996 - val_acc: 0.8838
Epoch 7/10
14044/14044 [==============================] - 5s - loss: 0.3632 - acc: 0.8924 - val_loss: 0.3561 - val_acc: 0.8975
Epoch 8/10
14044/14044 [==============================] - 5s -

In [35]:
# See how well it did on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Test loss: 0.327844048446
Test accuracy: 0.906236651004


In [36]:
# Save model for later usage
model.save('model.h5')